# Final Project for DTSA-5510

Kevin Juandi <br> 2023-11-25<br> https://github.com/Chiuchiyin/

## Introduction and Problem Description

Welcome to my final project for DTSA-5510! Before running this notebook, please ensure you have all required dependancies installed in your build. All required packages are listed in the next section. If you want to run the code, some parts of the code might take very long time to run. Please be patient with it. Thank you for reviewing my project!

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import cv2 as cv
import matplotlib.pyplot as plt
import os
import math
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
import time
import itertools
import random